In [1]:
import numpy as np
import glob
import os
import random
import re
from collections import Counter
import pandas as pd
import pickle

class_labels = ["CBFB_MYH11","control","NPM1","PML_RARA","RUNX1_RUNX1T1"]
n_classes = len(class_labels)

#Function to get list of image_paths in one folder
def get_image_path_list(folder_path):
    tif_files = glob.glob(f"{folder_path}/*.tif")
    return tif_files


#exracts the number of image in the file_path e.g. "image_123.tif"
def extract_number_image(file_path):
    # Use a regular expression to find the number after "image_" and before ".tif"
    match = re.search(r'image_(\d+).tif', file_path)
    return int(match.group(1))

def get_patient_name(path):
    return re.search(r"/data/\w+/([A-Z]{3})", path).group(1)

def get_class_name(path):
    return re.search(r"/data/(\w+)", path).group(1)

def get_classification_patient(patient_folder):
    probs_path = patient_folder + '/single_cell_probabilities.npy'
    sc_probs = np.load(probs_path)
    sc_class= np.argmax(sc_probs, axis=1)
    return sc_class


In [5]:
data_directory = '../../smalldataset/data'
subtype=data_directory+"/NPM1"
df = pd.DataFrame(columns=["patient","AML_subtype", "SC_Label", "image_path"])
# Save class classification count for each patient in csv file
for folder_class in os.listdir(data_directory):
    folder_class = os.path.join(data_directory, folder_class)
    #Skip folders not in class label
    if get_class_name(folder_class) not in class_labels:
        continue
    
    if os.path.isdir(folder_class):
       print(folder_class)
       AML_subtype=get_class_name(folder_class)
       for folder_patient in os.listdir(folder_class):
            folder_patient = os.path.join(folder_class, folder_patient)
            
            if os.path.isdir(folder_patient):
                images=get_image_path_list(folder_patient)
                sc_classes=get_classification_patient(folder_patient)

                for image in images:
                    number=extract_number_image(image)
                    df.loc[len(df)]=[get_patient_name(folder_patient), AML_subtype, sc_classes[number],image]

../../smalldataset/data/CBFB_MYH11


../../smalldataset/data/control
../../smalldataset/data/NPM1
../../smalldataset/data/PML_RARA
../../smalldataset/data/RUNX1_RUNX1T1


In [7]:
# Shuffle into n patients
n_patients=5
experiment_name = "experiment_3"

sc_class_labels= ['eosinophil granulocyte', 'reactive lymphocyte',
       'neutrophil granulocyte (segmented)', 'typical lymphocyte',
       'other', 'neutrophil granulocyte (band)', 'monocyte',
       'large granulated lymphocyte', 'atypical promyelocyte',
       'basophil granulocyte', 'smudge cell', 'neoplastic lymphocyte',
       'promyelocyte', 'myelocyte', 'myeloblast', 'metamyelocyte',
       'normo', 'plasma cell', 'hair cell', 'bilobed M3v',
       'mononucleosis']
#df_meanstd=pd.read_csv(data_directory+"/mean_std_sc_class.csv")
df_sc_res=pd.read_csv(data_directory+"/output/single_cell_results.csv",index_col=0).drop("patient", axis=1)
df_meanstd = df_sc_res.groupby(["AML_subtype"]).agg(["mean","std"])

In [8]:
patient_classes=[]
number_images=[]
output_folder = '../../artificialdata/'+experiment_name+'/data'
#Iterate over all AML subtypes 
for aml_subtype in class_labels:
    print(f"Generate patients for {aml_subtype}")
    output_class_folder=output_folder+"/"+aml_subtype
    #Get distribution
    class_means = df_meanstd.loc[aml_subtype,:].loc[:,"mean"].values
    class_variances = df_meanstd.loc[aml_subtype,:].loc[:,"std"].values
    for patient_number in range(n_patients):
        print(f"Patient {patient_number}")
        #calculate how many images from each sc class
        generated_data = np.random.normal(class_means,class_variances,21).astype(int)
        generated_data = generated_data*(generated_data>0)
        image_file_paths=[]
        #randomly choose sc-image and concatinate them into image_file_paths
        for cell_type_number, cell_type in enumerate(sc_class_labels):
            df_cell_type=df[df["SC_Label"]==cell_type_number]
            file_path=df_cell_type["image_path"].values
            image_file_paths+=(np.random.choice(file_path, size=generated_data[cell_type_number]).tolist())

        #Store patient classes and number images for meta data
        patient_classes.append(aml_subtype)
        number_images.append(len(image_file_paths))
        new_patient_folder = os.path.join(output_class_folder, f'patient_{patient_number+1}')
        os.makedirs(new_patient_folder, exist_ok=True)

        # Sort the shuffled image paths for the current patient
        image_file_paths.sort()
        # Save the shuffled image paths into a text file
        txt_file_path = os.path.join(new_patient_folder, 'images.txt')
        with open(txt_file_path, 'w') as txt_file:
            for image_path in image_file_paths:
                txt_file.write(image_path + '\n')

        with open( os.path.join(new_patient_folder,"image_file_paths") ,'wb') as fp:
            pickle.dump(image_file_paths,fp)
    

In [11]:
#Calculate and save feature matrix
for aml_subtype in class_labels:
    output_class_folder=output_folder+"/"+aml_subtype    
    for patient_number in range(n_patients):
        patient_folder = os.path.join(output_class_folder, f'patient_{patient_number+1}')
        with open( os.path.join(patient_folder,"image_file_paths") ,'rb') as fp:
            image_file_paths=pickle.load(fp)
        print(f"Save patient {patient_number + 1} features")
        array_list=[]
        previous_patient_id=None
        # Iterate through each image path
        for image_path in image_file_paths:
            patient_id = image_path[:image_path.find("/image")]
            if previous_patient_id!=patient_id:
                features=np.load(patient_id+"/fnl34_bn_features_layer_7.npy")
            array_list.append([features[extract_number_image(image_path)]])
            previous_patient_id=patient_id
        #Concatenate all features for one artificial patient    
        artificial_features = np.concatenate(array_list,axis=0)
        output_npy_file = output_folder+f"/{aml_subtype}/patient_{patient_number+1}/fnl34_bn_features_layer_7.npy"
        # Save the array to the .npy file
        np.save(output_npy_file, artificial_features)

Save patient 1 features
Save patient 2 features
Save patient 3 features
Save patient 4 features
Save patient 5 features
Save patient 1 features
Save patient 2 features
Save patient 3 features
Save patient 4 features
Save patient 5 features
Save patient 1 features
Save patient 2 features
Save patient 3 features
Save patient 4 features
Save patient 5 features
Save patient 1 features
Save patient 2 features
Save patient 3 features
Save patient 4 features
Save patient 5 features
Save patient 1 features
Save patient 2 features
Save patient 3 features
Save patient 4 features
Save patient 5 features


In [12]:
#Save metadata file
columns = ['patient_id', 'sex_1f_2m', 'age', 'bag_label', 'instance_count',
       'leucocytes_per_µl', 'pb_myeloblast', 'pb_promyelocyte',
       'pb_myelocyte', 'pb_metamyelocyte', 'pb_neutrophil_band',
       'pb_neutrophil_segmented', 'pb_eosinophil', 'pb_basophil',
       'pb_monocyte', 'pb_lymph_typ', 'pb_lymph_atyp_react',
       'pb_lymph_atyp_neopl', 'pb_other', 'pb_total']
artifcialmetadata = pd.DataFrame(columns=columns)
artifcialmetadata['patient_id'] = [f"patient_{i%5 + 1}" for i in range(n_patients*n_classes)]
artifcialmetadata['bag_label'] = patient_classes
artifcialmetadata['instance_count'] = number_images
artifcialmetadata.to_csv(os.path.dirname(output_folder)+'/metadata.csv', index=False)